This phase of capstone 2 kicks-off from the the previous phase of Capstone 2 EDA. Hence the csv file exported at the end of the EDA stage is re-imported here for feature engineering, data cleaning and training data development.

Objective
 - Data Preprocessing
 - one hot encode or create dummy variables
 - Standardize
 - train test split
 

In [2]:
# loading necessary modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


In [5]:
pv_df = pd.read_csv('pv_regression.csv', index_col = 0)
pv_df.head()

,installation_date,Year,system_size_DC,new_construction,ground_mounted,state,installer_name,DC_optimizer,dollar_per_watt,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module
0,2018-01-01,2018,10.3700,0.0,0.0,CA,Sunworks United,1.0,3.905497,18.711656,0,1,0
1,2018-01-01,2018,7.4305,0.0,0.0,CA,1st Light Energy,1.0,3.950707,16.512247,0,1,0
2,2018-01-01,2018,12.6000,0.0,0.0,CA,S E Lnstructionrporation,1.0,4.853175,20.917001,0,1,1
3,2018-01-01,2018,5.2800,0.0,0.0,CA,Sierra Pacific Home &Mfort,1.0,3.958333,19.710083,0,1,0
4,2018-01-01,2018,6.8400,0.0,0.0,CA,Syntrol Plumbing Heating & Air,1.0,4.991082,17.447199,0,1,0


In [6]:
pv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184778 entries, 0 to 184777
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   installation_date          184778 non-null  object 
 1   Year                       184778 non-null  int64  
 2   system_size_DC             184778 non-null  float64
 3   new_construction           184778 non-null  float64
 4   ground_mounted             184778 non-null  float64
 5   state                      184778 non-null  object 
 6   installer_name             184565 non-null  object 
 7   DC_optimizer               184778 non-null  float64
 8   dollar_per_watt            184778 non-null  float64
 9   Overall_Module_efficiency  184778 non-null  float64
 10  micro_inverter             184778 non-null  int64  
 11  installer_experience       184778 non-null  int64  
 12  Premium_Module             184778 non-null  int64  
dtypes: float64(6), int64(4), obje

In [12]:
pv_df['state'].value_counts()

CA    178722
MA      5751
NH       189
FL       116
Name: state, dtype: int64

In [14]:
pv_df['Premium_Module'].value_counts()

0    123182
1     61596
Name: Premium_Module, dtype: int64

In [15]:
pv_df['installer_name'].value_counts()

Tesla Energy                   17313
Sunpower Capital                9685
Sunrun                          9185
Semper Solarisnstruction        6240
Petersen Dean                   5733
                               ...  
Lovenstruction                     1
Ra Power Solar And Electric        1
Swg Energy                         1
Arm Hvac                           1
Art Ofnstruction                   1
Name: installer_name, Length: 3034, dtype: int64

In [16]:
pv_df['DC_optimizer'].value_counts()

0.0    95861
1.0    88917
Name: DC_optimizer, dtype: int64

A quick recap of the data frame above shows that categoriacal variables are - 
 - Installation_date
 - State
 - Installer Name
 
 Of the three we will encode the State. The installation date column was mainly used to capture the year of installation and since its actually a date_time type, it can be dropped. 
 
 Additionally the installation date and Installer name are mainly used to calculate the installer experience and have hence served their purpose.

In [19]:
pv_df1 = pv_df.drop(columns = ['installation_date','installer_name'], axis = 1)
pv_df1.head()

,Year,system_size_DC,new_construction,ground_mounted,state,DC_optimizer,dollar_per_watt,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module
0,2018,10.3700,0.0,0.0,CA,1.0,3.905497,18.711656,0,1,0
1,2018,7.4305,0.0,0.0,CA,1.0,3.950707,16.512247,0,1,0
2,2018,12.6000,0.0,0.0,CA,1.0,4.853175,20.917001,0,1,1
3,2018,5.2800,0.0,0.0,CA,1.0,3.958333,19.710083,0,1,0
4,2018,6.8400,0.0,0.0,CA,1.0,4.991082,17.447199,0,1,0


In [20]:
pv_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184778 entries, 0 to 184777
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Year                       184778 non-null  int64  
 1   system_size_DC             184778 non-null  float64
 2   new_construction           184778 non-null  float64
 3   ground_mounted             184778 non-null  float64
 4   state                      184778 non-null  object 
 5   DC_optimizer               184778 non-null  float64
 6   dollar_per_watt            184778 non-null  float64
 7   Overall_Module_efficiency  184778 non-null  float64
 8   micro_inverter             184778 non-null  int64  
 9   installer_experience       184778 non-null  int64  
 10  Premium_Module             184778 non-null  int64  
dtypes: float64(6), int64(4), object(1)
memory usage: 16.9+ MB


Note here the installer experience is the substitute for installer name. 
Moving forward - we will extract the state column and create a dummy variable for the state feature, concat it back to the main dataframe and subsequently remove the original categorical featire of the state. 

In [22]:
pv_df2 = pd.get_dummies(pv_df1)
pv_df2.head()

,Year,system_size_DC,new_construction,ground_mounted,DC_optimizer,dollar_per_watt,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module,state_CA,state_FL,state_MA,state_NH
0,2018,10.3700,0.0,0.0,1.0,3.905497,18.711656,0,1,0,1,0,0,0
1,2018,7.4305,0.0,0.0,1.0,3.950707,16.512247,0,1,0,1,0,0,0
2,2018,12.6000,0.0,0.0,1.0,4.853175,20.917001,0,1,1,1,0,0,0
3,2018,5.2800,0.0,0.0,1.0,3.958333,19.710083,0,1,0,1,0,0,0
4,2018,6.8400,0.0,0.0,1.0,4.991082,17.447199,0,1,0,1,0,0,0


In [23]:
pv_df1.head()

,Year,system_size_DC,new_construction,ground_mounted,state,DC_optimizer,dollar_per_watt,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module
0,2018,10.3700,0.0,0.0,CA,1.0,3.905497,18.711656,0,1,0
1,2018,7.4305,0.0,0.0,CA,1.0,3.950707,16.512247,0,1,0
2,2018,12.6000,0.0,0.0,CA,1.0,4.853175,20.917001,0,1,1
3,2018,5.2800,0.0,0.0,CA,1.0,3.958333,19.710083,0,1,0
4,2018,6.8400,0.0,0.0,CA,1.0,4.991082,17.447199,0,1,0


In [24]:
pv_df2 = pv_df2.astype({"new_construction":'int', "ground_mounted":'int', "DC_optimizer" : 'int'})
pv_df2.head()

,Year,system_size_DC,new_construction,ground_mounted,DC_optimizer,dollar_per_watt,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module,state_CA,state_FL,state_MA,state_NH
0,2018,10.3700,0,0,1,3.905497,18.711656,0,1,0,1,0,0,0
1,2018,7.4305,0,0,1,3.950707,16.512247,0,1,0,1,0,0,0
2,2018,12.6000,0,0,1,4.853175,20.917001,0,1,1,1,0,0,0
3,2018,5.2800,0,0,1,3.958333,19.710083,0,1,0,1,0,0,0
4,2018,6.8400,0,0,1,4.991082,17.447199,0,1,0,1,0,0,0


In [25]:
pv_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184778 entries, 0 to 184777
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Year                       184778 non-null  int64  
 1   system_size_DC             184778 non-null  float64
 2   new_construction           184778 non-null  int64  
 3   ground_mounted             184778 non-null  int64  
 4   DC_optimizer               184778 non-null  int64  
 5   dollar_per_watt            184778 non-null  float64
 6   Overall_Module_efficiency  184778 non-null  float64
 7   micro_inverter             184778 non-null  int64  
 8   installer_experience       184778 non-null  int64  
 9   Premium_Module             184778 non-null  int64  
 10  state_CA                   184778 non-null  uint8  
 11  state_FL                   184778 non-null  uint8  
 12  state_MA                   184778 non-null  uint8  
 13  state_NH                   18

In [26]:
pv_df2['Year'].value_counts()

2019    93266
2018    91512
Name: Year, dtype: int64

Since we are explicitly focusing on installations from the year 2018 and 2019 we can drop the Year column. 

In [28]:
pv_df2.drop(columns = 'Year', axis = 1, inplace = True)


KeyError: "['Year'] not found in axis"

In [29]:
pv_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184778 entries, 0 to 184777
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   system_size_DC             184778 non-null  float64
 1   new_construction           184778 non-null  int64  
 2   ground_mounted             184778 non-null  int64  
 3   DC_optimizer               184778 non-null  int64  
 4   dollar_per_watt            184778 non-null  float64
 5   Overall_Module_efficiency  184778 non-null  float64
 6   micro_inverter             184778 non-null  int64  
 7   installer_experience       184778 non-null  int64  
 8   Premium_Module             184778 non-null  int64  
 9   state_CA                   184778 non-null  uint8  
 10  state_FL                   184778 non-null  uint8  
 11  state_MA                   184778 non-null  uint8  
 12  state_NH                   184778 non-null  uint8  
dtypes: float64(3), int64(6), uint

In [32]:
# renaming columns for ease of use
pv_df2.rename(columns = {'system_size_DC': 'kW'}, inplace = True)

In [33]:
pv_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184778 entries, 0 to 184777
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   kW                         184778 non-null  float64
 1   new_construction           184778 non-null  int64  
 2   ground_mounted             184778 non-null  int64  
 3   DC_optimizer               184778 non-null  int64  
 4   dollar_per_watt            184778 non-null  float64
 5   Overall_Module_efficiency  184778 non-null  float64
 6   micro_inverter             184778 non-null  int64  
 7   installer_experience       184778 non-null  int64  
 8   Premium_Module             184778 non-null  int64  
 9   state_CA                   184778 non-null  uint8  
 10  state_FL                   184778 non-null  uint8  
 11  state_MA                   184778 non-null  uint8  
 12  state_NH                   184778 non-null  uint8  
dtypes: float64(3), int64(6), uint

In [34]:
# considering only the numerical columns for standardization
from sklearn.preprocessing import StandardScaler

X = pv_df2.drop(columns = 'dollar_per_watt', axis = 1)
y = pv_df2['dollar_per_watt']



In [35]:
X.head()

,kW,new_construction,ground_mounted,DC_optimizer,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module,state_CA,state_FL,state_MA,state_NH
0,10.3700,0,0,1,18.711656,0,1,0,1,0,0,0
1,7.4305,0,0,1,16.512247,0,1,0,1,0,0,0
2,12.6000,0,0,1,20.917001,0,1,1,1,0,0,0
3,5.2800,0,0,1,19.710083,0,1,0,1,0,0,0
4,6.8400,0,0,1,17.447199,0,1,0,1,0,0,0


In [36]:
y.head()

0    3.905497
1    3.950707
2    4.853175
3    3.958333
4    4.991082
Name: dollar_per_watt, dtype: float64

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [39]:
X_train.head()

,kW,new_construction,ground_mounted,DC_optimizer,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module,state_CA,state_FL,state_MA,state_NH
106981,5.90,0,0,1,18.198822,0,15,0,1,0,0,0
77562,6.71,0,0,1,18.263473,0,93,0,1,0,0,0
64911,7.70,0,0,1,19.743590,0,87,0,1,0,0,0
36565,9.57,0,0,1,19.710083,0,54,0,1,0,0,0
26404,9.54,0,0,0,16.462286,1,124,0,1,0,0,0


In [40]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129344 entries, 106981 to 15725
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   kW                         129344 non-null  float64
 1   new_construction           129344 non-null  int64  
 2   ground_mounted             129344 non-null  int64  
 3   DC_optimizer               129344 non-null  int64  
 4   Overall_Module_efficiency  129344 non-null  float64
 5   micro_inverter             129344 non-null  int64  
 6   installer_experience       129344 non-null  int64  
 7   Premium_Module             129344 non-null  int64  
 8   state_CA                   129344 non-null  uint8  
 9   state_FL                   129344 non-null  uint8  
 10  state_MA                   129344 non-null  uint8  
 11  state_NH                   129344 non-null  uint8  
dtypes: float64(2), int64(6), uint8(4)
memory usage: 9.4 MB


In [41]:
# extracting only numeric features in both the train set before applying standard scaler
# apply the scaler fit and transform to the X_tr dataset
X_tr = X_train[['kW', 'Overall_Module_efficiency', 'installer_experience']]

In [42]:
X_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129344 entries, 106981 to 15725
Data columns (total 3 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   kW                         129344 non-null  float64
 1   Overall_Module_efficiency  129344 non-null  float64
 2   installer_experience       129344 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 3.9 MB


In [43]:
# extracting only numeric features in the test set.
# Apply the scaler transformation to X_te
X_te = X_test[['kW', 'Overall_Module_efficiency', 'installer_experience']]

In [44]:
scaler = StandardScaler()
X_tr_scaled = scaler.fit(X_tr) # fitting only numeric features from train dataset


In [45]:
X_tr_scaled.mean_

array([   7.26369865,   19.40001248, 1785.56053624])

In [46]:
print(X_tr_scaled)

StandardScaler()


In [48]:
X_tr_transformed = X_tr_scaled.transform(X_tr) # transforming training data set using the scaler

In [49]:
X_tr_transformed.mean(axis = 0)

array([-7.36394836e-17,  4.77709026e-16,  2.74671703e-18])

In [50]:
# transforming the test data set using the scaler created from the train data set
X_te_transformed = X_tr_scaled.transform(X_te)

In [51]:
X_te_transformed.mean(axis = 0)

array([ 5.89268800e-05, -5.56999124e-03, -2.57322367e-03])

In [52]:
# putting the transformed values in train data sset back to the original train dataset
X_train.head()

,kW,new_construction,ground_mounted,DC_optimizer,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module,state_CA,state_FL,state_MA,state_NH
106981,5.90,0,0,1,18.198822,0,15,0,1,0,0,0
77562,6.71,0,0,1,18.263473,0,93,0,1,0,0,0
64911,7.70,0,0,1,19.743590,0,87,0,1,0,0,0
36565,9.57,0,0,1,19.710083,0,54,0,1,0,0,0
26404,9.54,0,0,0,16.462286,1,124,0,1,0,0,0


In [54]:
X_train.loc[:, ['kW', 'Overall_Module_efficiency', 'installer_experience']] = X_tr_transformed

In [55]:
X_train.head()

,kW,new_construction,ground_mounted,DC_optimizer,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module,state_CA,state_FL,state_MA,state_NH
106981,-0.227901,0,0,1,-0.806933,0,-0.548542,0,1,0,0,0
77562,-0.092534,0,0,1,-0.763502,0,-0.524376,0,1,0,0,0
64911,0.072915,0,0,1,0.230808,0,-0.526235,0,1,0,0,0
36565,0.385429,0,0,1,0.208298,0,-0.536459,0,1,0,0,0
26404,0.380415,0,0,0,-1.973500,1,-0.514772,0,1,0,0,0


In [57]:
X_tr_transformed

array([[-0.22790129, -0.80693287, -0.54854168],
       [-0.09253411, -0.76350202, -0.52437631],
       [ 0.07291467,  0.2308076 , -0.52623518],
       ...,
       [-0.55879885, -1.42857181, -0.55009074],
       [ 0.03625741, -1.41333716, -0.40850646],
       [-0.39919927,  0.00768011,  0.04134121]])

In [59]:
X_test.loc[:, ['kW', 'Overall_Module_efficiency', 'installer_experience']] = X_te_transformed

In [60]:
X_test.head()

,kW,new_construction,ground_mounted,DC_optimizer,Overall_Module_efficiency,micro_inverter,installer_experience,Premium_Module,state_CA,state_FL,state_MA,state_NH
93532,-0.127629,0,0,1,-2.083261,0,-0.548851,0,1,0,0,0
175019,-0.304776,0,0,1,0.821078,0,0.687610,1,1,0,0,0
52621,0.169844,0,0,0,1.797817,1,0.002924,1,1,0,0,0
184169,-0.055768,0,0,0,0.822919,1,-0.441037,1,1,0,0,0
144367,0.230007,0,1,0,1.804313,1,-0.548851,1,1,0,0,0
